In [19]:
import pandas as pd
import numpy as np

Я заниюмаюсь управлением товародвижения в федеральной сети ювелирных магазинов (900 магазинов). Отвечаю за изделия из золота. В данной работе хочу применить машинное обучение для:

- прогноза продаж ассортимента определенного типа для каждого торгового объекта(магазина) в отдельности
- выявление главных признаков торгового объекта(магазина), котороые влияют на продажи ассортимента определенного типа

Делать это буду на примере мусульманских подвесов.
Для анализа буду использовать данные по продажам мусульманских подвесов из серебра, т.к. они присутствуют почти в каждом магазине.
Практическая польза

In [20]:
# Загружаю датасет с продажами серебра и золота

mus_sales = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='олап', skiprows=10, usecols=[0,2,3], names=['id','585_msale','925_msale'])

# Убираем пустые значения после 'Общий итог'
lst = mus_sales['id'].tolist()
mus_sales = mus_sales.drop(axis=0,index=np.arange(lst.index('Общий итог'),len(lst)))
mus_sales.tail()

,id,585_msale,925_msale
852,91,1.0,1.0
853,925,0.0,2.0
854,955,0.0,6.0
855,96,3.0,10.0
856,99,6.0,8.0


In [21]:
mus_stock = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='олап', skiprows=10, usecols=[7,9,10],names=['id','585_mstock','925_mstock'])

# Убираем пустые значения
lst = mus_stock['id'].tolist()
mus_stock = mus_stock.drop(axis=0,index=np.arange(lst.index('Общий итог'),len(lst)))

# Убираем магазины с нулевыми остатками
mus_stock['sum'] = mus_stock['585_mstock'] + mus_stock['925_mstock']
mus_stock = mus_stock[(mus_stock['sum'] > 0)]
del mus_stock['sum']
mus_stock.tail()

,id,585_mstock,925_mstock
1426,92,16.0,8.0
1427,925,1.0,8.0
1428,955,6.0,12.0
1429,96,15.0,7.0
1432,99,25.0,15.0


In [24]:
# Оставляем склады, где есть остатки золото и серебро одновременно

mus_stock_combined = mus_stock[(mus_stock['585_mstock'] > 5)&(mus_stock['925_mstock'] > 5)].drop(axis=1, labels=['585_mstock','925_mstock'])

# Посмотрим корреляцию в продажах золота и серебра
mus_sales_combined = pd.merge(mus_stock_combined, mus_sales, on='id',how='left').dropna(how='any')
mus_sales_combined[['585_msale','925_msale']].corr().round(decimals=1)

,585_msale,925_msale
585_msale,1.0,0.7
925_msale,0.7,1.0


Коэффицент корреляции между золотом и серебром 0,7. Можно использовать данные по серебру

In [12]:
# Посмотрим есть ли разница в распределении продаж по золоту и серебру
mus_sales_combined.describe().round()

,585_msale,925_msale
count,468.0,468.0
mean,7.0,8.0
std,7.0,8.0
min,0.0,0.0
25%,2.0,3.0
50%,4.0,6.0
75%,10.0,10.0
max,39.0,60.0


Вопросы: 
нужно ли объединять тц и трц?


Создаем большой датасет для обучения модели со следующей информацией:

type - тип размещения объекта (отдельное здание или торговый центр)
city - город или населенный пункт, по этому параметру каждому объекту можно установить признак региона
shop_size - вместимость объекта, этот параметр разобью на 5 размерных групп
work_time - время работы объекта в годах - разобью на две группы до и более 2 лет
total_sales - общие продажи в штуках за полгода
brand - вывеска на магазине(есть несколько брэндов)
population - количество жителей в данном населенном пункте разобью на 5 размерных групп
salary - средняя зарплата в регионе разобью на 5 размерных групп
mosque - есть или нет в населенном пункте мечеть(по данным из википедии)
muslim_persentage - процент верующих мусульман в регионе(не в населенном пункте) - разобью на 2-3 группы

In [38]:
# Загружаем справочник с характеристиками магазинов и дополнительной информацией по регионам. 
ref_book = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='типы', usecols=[0,5,7,9,10,11,12,13,14,15,16,17])
ref_book['id'] = ref_book['id'].astype(str)
ref_book.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903 entries, 0 to 902
Data columns (total 12 columns):
type                 882 non-null object
city                 903 non-null object
id                   903 non-null object
shop_size            900 non-null float64
work_time            903 non-null int64
total_sales          899 non-null float64
stock_month          903 non-null int64
brand                903 non-null object
population           0 non-null float64
salary               0 non-null float64
mosque               903 non-null int64
muslim_persentage    0 non-null float64
dtypes: float64(5), int64(3), object(4)
memory usage: 84.7+ KB


In [42]:
# Добавляем к продажам подвесов по магазинам всю информацию из справочника

raw_df = pd.merge(mus_sales[['id','925_msale']],ref_book,how='left', on='id')
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 859 entries, 0 to 858
Data columns (total 13 columns):
id                   859 non-null object
925_msale            859 non-null float64
type                 828 non-null object
city                 845 non-null object
shop_size            845 non-null float64
work_time            845 non-null float64
total_sales          845 non-null float64
stock_month          845 non-null float64
brand                845 non-null object
population           0 non-null float64
salary               0 non-null float64
mosque               845 non-null float64
muslim_persentage    0 non-null float64
dtypes: float64(9), object(4)
memory usage: 94.0+ KB


очищаем от магазинов с маленькими нормами, с отсутствием продаж по золоту

In [33]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Yandex") 
location = geolocator.geocode("Чегем ")
print(location)

Чегем, городское поселение Чегем, Чегемский район, Кабардино-Балкария, СКФО, РФ


In [27]:
(geolocator.geocode("Владикавказ, РФ").latitude, geolocator.geocode("Владикавказ, РФ").longitude)

(43.024593, 44.68211)

In [149]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Google")
location = geolocator.geocode(lst[0:3])
location.address

AttributeError: 'NoneType' object has no attribute 'address'